In [213]:
# import math
# import numpy as np
# import pandas as pd
# from sklearn.grid_search import GridSearchCV
# from sklearn.linear_model import LogisticRegressionCV
# from sklearn.naive_bayes import GaussianNB
# from sklearn.cross_validation import cross_val_score, cross_val_predict, StratifiedKFold 
# from sklearn import preprocessing, metrics, svm, ensemble
# from sklearn.metrics import accuracy_score, classification_report
import tabpy_client 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import itertools

pd.plotting.register_matplotlib_converters()

import datetime
import time

from statsmodels.tsa.seasonal import seasonal_decompose
import statsmodels.tsa.stattools as ts
import statsmodels.api as sm

import datetime

from math import sqrt

from fbprophet import Prophet

from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error,accuracy_score

In [3]:
emergency_df = pd.read_pickle('./emergency_df')


# Take a look at the structure of the file
emergency_df.head(n=10)

,RecordID,CallNumber,CallDateTime,Priority,District,Description,IncidentLocation,ZipCode,Neighborhood,PoliceDistrict,PolicePost,CouncilDistrict,SheriffDistricts,Community_Statistical_Areas,Census_Tracts,Location
0,2333660,P163072128,11/02/2016 04:46:00 PM,Medium,SD,NARCOTICSOutside,1200 W CROSS ST,21230.0,Washington Village/Pigtown,Southern,932.0,10.0,D7,Washington Village/Pigtown,Census Tract 2102,"1200 W CROSS ST\nBALTIMORE, MD\n(39.281995, -7..."
1,2334887,P163070732,11/02/2016 09:22:00 AM,High,SW,Traffic Stop,2100 W PRATT ST,21223.0,Carrollton Ridge,Southwestern,841.0,9.0,D8,Southwest Baltimore,Census Tract 2005,"2100 W PRATT ST\nBALTIMORE, MD\n(39.284965, -7..."
2,2334245,P163071480,11/02/2016 01:37:00 PM,Low,SE,LARCENY F/AUTO,HOME DEPOT,21224.0,Pulaski Industrial Area,Southeastern,242.0,1.0,D6,Orangeville/East Highlandtown,Census Tract 2605.01,"HOME DEPOT BALTIMORE, MD\n(39.499699, -76.641403)"
3,2332907,P163072974,11/02/2016 08:20:00 PM,Medium,ND,911/NO VOICE,5200 YORK RD,21212.0,York-Homeland,Northern,521.0,4.0,D2,North Baltimore/Guilford/Homeland,Census Tract 2711.01,"5200 YORK RD\nBALTIMORE, MD\n(39.352315, -76.6..."
4,2332699,P163073203,11/02/2016 09:39:00 PM,Medium,NW,COMMON ASSAULT,2500 N ELLAMONT ST,21216.0,Hanlon-Longwood,Northwestern,641.0,7.0,D9,Greater Mondawmin,Census Tract 1507.01,"2500 N ELLAMONT ST\nBALTIMORE, MD\n(39.315441,..."
5,2483708,P162552312,09/11/2016 06:46:00 PM,Medium,SD,911/NO VOICE,600 CHERRY HILL RD,21225.0,Cherry Hill,Southern,922.0,10.0,D7,Cherry Hill,Census Tract 2502.07,"600 CHERRY HILL RD\nBALTIMORE, MD\n(39.250045,..."
6,2484088,P162551897,09/11/2016 04:31:00 PM,Medium,ND,911/NO VOICE,2500 W BELVEDERE AV,21215.0,Levindale,Northern,532.0,5.0,D2,Pimlico/Arlington/Hilltop,Census Tract 2717,"2500 W BELVEDERE AV\nBALTIMORE, MD\n(39.352908..."
7,2482512,P162560534,09/12/2016 07:49:00 AM,Low,ND,INVEST,900 W LAKE AV,21210.0,North Roland Park/Poplar Hill,Northern,521.0,5.0,D2,Greater Roland Park/Poplar Hill,Census Tract 2713,"900 W LAKE AV\nBALTIMORE, MD\n(39.373542, -76...."
8,2483324,P162552725,09/11/2016 09:09:00 PM,Medium,SW,911/NO VOICE,3600 W FRANKLIN ST,21229.0,Allendale,Southwestern,843.0,8.0,D10,Allendale/Irvington/S. Hilton,Census Tract 2007.01,"3600 W FRANKLIN ST\nBALTIMORE, MD\n(39.292646,..."
9,2484350,P162551605,09/11/2016 02:46:00 PM,Medium,NW,DISORDERLY,5500 PARK HEIGHTS AV,21215.0,Pimlico Good Neighbors,Northwestern,633.0,5.0,D1,Pimlico/Arlington/Hilltop,Census Tract 2717,"5500 PARK HEIGHTS AV\nBALTIMORE, MD\n(39.3519,..."


In [4]:
emergency_df.drop(emergency_df[(emergency_df.District == 'TRU') | (emergency_df.District == 'CW')\
                               | (emergency_df.District == 'SS') | (emergency_df.District == 'HP')\
                               | (emergency_df.District == 'EVT1') | (emergency_df.District == 'INFO')\
                               | (emergency_df.District == 'EVT1') | (emergency_df.District == 'EVT2')\
                               | (emergency_df.District == 'EVT3') | (emergency_df.District == 'FIR1')]\
                  .index, inplace=True)
emergency_df.reset_index(drop=True, inplace=True)

In [17]:
def agg_df(df, mdy):
    new_df = pd.DataFrame()
    new_df['CallDateTime'] = df.loc[df["CallDateTime"].notnull(), "CallDateTime"].copy(deep=True)

    # #convert to time series
    new_df["CallDateTime"] = new_df["CallDateTime"].apply(pd.to_datetime)
    new_df = new_df.set_index(new_df["CallDateTime"])

    # # Group by month = M, by week = W, by day = D
    grouped = new_df.resample(mdy).count()
    new_df = pd.DataFrame({"count": grouped.values.flatten()}, index=grouped.index)
    #new_df['District'] = district
    return (new_df)

In [5]:
#calls_df = pd.DataFrame(columns=emergency_df.columns)

In [16]:
call_df = pd.DataFrame()
districts = emergency_df['District'].unique()
for i, district in enumerate(districts):
    dist_df = emergency_df[(emergency_df.District == district)]
    dist_df.reset_index(drop=True, inplace=True)
    dist_df = agg_df(dist_df, "M", district)
    if i == 0:
        calls_df = dist_df
    else:
        calls_df = calls_df.merge(dist_df, 'outer')
calls_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 698 entries, 0 to 697
Data columns (total 2 columns):
count       698 non-null int64
District    698 non-null object
dtypes: int64(1), object(1)
memory usage: 16.4+ KB


In [9]:
calls_df = pd.DataFrame()

In [11]:
calls_df = pd.DataFrame(columns= ['count', 'District'])
calls_df

,count,District


In [13]:
calls_df.merge(dist_df, 'outer')

,count,District
0,1,SW
1,10125,SW
2,10608,SW
3,9723,SW
4,9965,SW
...,...,...
73,3207,SW
74,0,SW
75,9055,SW
76,14542,SW


In [ ]:
# Create df for each district

In [18]:
calls_SD_df = emergency_df[(emergency_df.District == 'SD')]
calls_SD_df.reset_index(drop=True, inplace=True)
calls_SD_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 694727 entries, 0 to 694726
Data columns (total 16 columns):
RecordID                       694727 non-null int64
CallNumber                     694727 non-null object
CallDateTime                   694727 non-null object
Priority                       694727 non-null object
District                       694727 non-null object
Description                    694727 non-null object
IncidentLocation               694727 non-null object
ZipCode                        694727 non-null float64
Neighborhood                   694727 non-null object
PoliceDistrict                 694727 non-null object
PolicePost                     694727 non-null float64
CouncilDistrict                694727 non-null float64
SheriffDistricts               694727 non-null object
Community_Statistical_Areas    694727 non-null object
Census_Tracts                  694727 non-null object
Location                       694727 non-null object
dtypes: float64(3), int

In [19]:
calls_SW_df = emergency_df[(emergency_df.District == 'SW')]
calls_SW_df.reset_index(drop=True, inplace=True)
calls_SW_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 715117 entries, 0 to 715116
Data columns (total 16 columns):
RecordID                       715117 non-null int64
CallNumber                     715117 non-null object
CallDateTime                   715117 non-null object
Priority                       715117 non-null object
District                       715117 non-null object
Description                    715117 non-null object
IncidentLocation               715117 non-null object
ZipCode                        715117 non-null float64
Neighborhood                   715117 non-null object
PoliceDistrict                 715117 non-null object
PolicePost                     715117 non-null float64
CouncilDistrict                715117 non-null float64
SheriffDistricts               715117 non-null object
Community_Statistical_Areas    715117 non-null object
Census_Tracts                  715117 non-null object
Location                       715117 non-null object
dtypes: float64(3), int

In [20]:
calls_SE_df = emergency_df[(emergency_df.District == 'SE')]
calls_SE_df.reset_index(drop=True, inplace=True)
calls_SE_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 689590 entries, 0 to 689589
Data columns (total 16 columns):
RecordID                       689590 non-null int64
CallNumber                     689590 non-null object
CallDateTime                   689590 non-null object
Priority                       689590 non-null object
District                       689590 non-null object
Description                    689590 non-null object
IncidentLocation               689590 non-null object
ZipCode                        689590 non-null float64
Neighborhood                   689590 non-null object
PoliceDistrict                 689590 non-null object
PolicePost                     689590 non-null float64
CouncilDistrict                689590 non-null float64
SheriffDistricts               689590 non-null object
Community_Statistical_Areas    689590 non-null object
Census_Tracts                  689590 non-null object
Location                       689590 non-null object
dtypes: float64(3), int

In [21]:
calls_ND_df = emergency_df[(emergency_df.District == 'ND')]
calls_ND_df.reset_index(drop=True, inplace=True)
calls_ND_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 653535 entries, 0 to 653534
Data columns (total 16 columns):
RecordID                       653535 non-null int64
CallNumber                     653535 non-null object
CallDateTime                   653535 non-null object
Priority                       653535 non-null object
District                       653535 non-null object
Description                    653535 non-null object
IncidentLocation               653535 non-null object
ZipCode                        653535 non-null float64
Neighborhood                   653535 non-null object
PoliceDistrict                 653535 non-null object
PolicePost                     653535 non-null float64
CouncilDistrict                653535 non-null float64
SheriffDistricts               653535 non-null object
Community_Statistical_Areas    653535 non-null object
Census_Tracts                  653535 non-null object
Location                       653535 non-null object
dtypes: float64(3), int

In [22]:
calls_NW_df = emergency_df[(emergency_df.District == 'NW')]
calls_NW_df.reset_index(drop=True, inplace=True)
calls_NW_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 639622 entries, 0 to 639621
Data columns (total 16 columns):
RecordID                       639622 non-null int64
CallNumber                     639622 non-null object
CallDateTime                   639622 non-null object
Priority                       639622 non-null object
District                       639622 non-null object
Description                    639622 non-null object
IncidentLocation               639622 non-null object
ZipCode                        639622 non-null float64
Neighborhood                   639622 non-null object
PoliceDistrict                 639622 non-null object
PolicePost                     639622 non-null float64
CouncilDistrict                639622 non-null float64
SheriffDistricts               639622 non-null object
Community_Statistical_Areas    639622 non-null object
Census_Tracts                  639622 non-null object
Location                       639622 non-null object
dtypes: float64(3), int

In [23]:
calls_ED_df = emergency_df[(emergency_df.District == 'ED')]
calls_ED_df.reset_index(drop=True, inplace=True)
calls_ED_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 598424 entries, 0 to 598423
Data columns (total 16 columns):
RecordID                       598424 non-null int64
CallNumber                     598424 non-null object
CallDateTime                   598424 non-null object
Priority                       598424 non-null object
District                       598424 non-null object
Description                    598424 non-null object
IncidentLocation               598424 non-null object
ZipCode                        598424 non-null float64
Neighborhood                   598424 non-null object
PoliceDistrict                 598424 non-null object
PolicePost                     598424 non-null float64
CouncilDistrict                598424 non-null float64
SheriffDistricts               598424 non-null object
Community_Statistical_Areas    598424 non-null object
Census_Tracts                  598424 non-null object
Location                       598424 non-null object
dtypes: float64(3), int

In [24]:
calls_CD_df = emergency_df[(emergency_df.District == 'CD')]
calls_CD_df.reset_index(drop=True, inplace=True)
calls_CD_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 694923 entries, 0 to 694922
Data columns (total 16 columns):
RecordID                       694923 non-null int64
CallNumber                     694923 non-null object
CallDateTime                   694923 non-null object
Priority                       694923 non-null object
District                       694923 non-null object
Description                    694923 non-null object
IncidentLocation               694923 non-null object
ZipCode                        694923 non-null float64
Neighborhood                   694923 non-null object
PoliceDistrict                 694923 non-null object
PolicePost                     694923 non-null float64
CouncilDistrict                694923 non-null float64
SheriffDistricts               694923 non-null object
Community_Statistical_Areas    694923 non-null object
Census_Tracts                  694923 non-null object
Location                       694923 non-null object
dtypes: float64(3), int

In [25]:
calls_WD_df = emergency_df[(emergency_df.District == 'WD')]
calls_WD_df.reset_index(drop=True, inplace=True)
calls_WD_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 654725 entries, 0 to 654724
Data columns (total 16 columns):
RecordID                       654725 non-null int64
CallNumber                     654725 non-null object
CallDateTime                   654725 non-null object
Priority                       654725 non-null object
District                       654725 non-null object
Description                    654725 non-null object
IncidentLocation               654725 non-null object
ZipCode                        654725 non-null float64
Neighborhood                   654725 non-null object
PoliceDistrict                 654725 non-null object
PolicePost                     654725 non-null float64
CouncilDistrict                654725 non-null float64
SheriffDistricts               654725 non-null object
Community_Statistical_Areas    654725 non-null object
Census_Tracts                  654725 non-null object
Location                       654725 non-null object
dtypes: float64(3), int

In [26]:
calls_NE_df = emergency_df[(emergency_df.District == 'NE')]
calls_NE_df.reset_index(drop=True, inplace=True)
calls_NE_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 905331 entries, 0 to 905330
Data columns (total 16 columns):
RecordID                       905331 non-null int64
CallNumber                     905331 non-null object
CallDateTime                   905331 non-null object
Priority                       905331 non-null object
District                       905331 non-null object
Description                    905331 non-null object
IncidentLocation               905331 non-null object
ZipCode                        905331 non-null float64
Neighborhood                   905331 non-null object
PoliceDistrict                 905331 non-null object
PolicePost                     905331 non-null float64
CouncilDistrict                905331 non-null float64
SheriffDistricts               905331 non-null object
Community_Statistical_Areas    905331 non-null object
Census_Tracts                  905331 non-null object
Location                       905331 non-null object
dtypes: float64(3), int

In [ ]:
# resample to create df for daily predictions

In [27]:
calls_d_SD_df = agg_df(calls_SD_df, "D")

In [28]:
calls_d_SW_df = agg_df(calls_SW_df, "D")

In [29]:
calls_d_SE_df = agg_df(calls_SE_df, "D")

In [30]:
calls_d_ND_df = agg_df(calls_ND_df, "D")

In [31]:
calls_d_NW_df = agg_df(calls_NW_df, "D")

In [32]:
calls_d_ED_df = agg_df(calls_ED_df, "D")

In [33]:
calls_d_CD_df = agg_df(calls_CD_df, "D")

In [34]:
calls_d_WD_df = agg_df(calls_WD_df, "D")

In [35]:
calls_d_NE_df = agg_df(calls_NE_df, "D")

,RecordID,CallNumber,CallDateTime,Priority,District,Description,IncidentLocation,ZipCode,Neighborhood,PoliceDistrict,PolicePost,CouncilDistrict,SheriffDistricts,Community_Statistical_Areas,Census_Tracts,Location
0,2483220,P162552835,09/11/2016 09:51:00 PM,Medium,CD,JUV DISTURBANCE,500 W MULBERRY ST,21201.0,University Of Maryland,Central,121.0,11.0,D8,Downtown/Seton Hill,Census Tract 402,"500 W MULBERRY ST\nBALTIMORE, MD\n(39.293797, ..."
1,2482779,P162560223,09/12/2016 02:49:00 AM,High,CD,PROWLER,600 W FRANKLIN ST,21201.0,Seton Hill,Central,143.0,11.0,D8,Downtown/Seton Hill,Census Tract 1701,"600 W FRANKLIN ST\nBALTIMORE, MD\n(39.294706, ..."
2,2483902,P162552100,09/11/2016 05:36:00 PM,Low,CD,WANTED ON WARR,500 E BALTIMORE ST,21202.0,Downtown,Central,111.0,11.0,D7,Downtown/Seton Hill,Census Tract 401,"500 E BALTIMORE ST\nBALTIMORE, MD\n(39.289802,..."
3,2483615,P162552630,09/11/2016 07:24:00 PM,Low,CD,SEE TEXT,300 W FAYETTE ST,21201.0,Downtown,Central,111.0,11.0,D7,Downtown/Seton Hill,Census Tract 401,"300 W FAYETTE ST\nBALTIMORE, MD\n(39.290455, -..."
4,2334452,P163071256,11/02/2016 12:17:00 PM,Low,CD,LARCENY,200 E REDWOOD ST,21202.0,Downtown,Central,111.0,11.0,D7,Downtown/Seton Hill,Census Tract 401,"200 E REDWOOD ST\nBALTIMORE, MD\n(39.288932, -..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
694918,5842844,132112146,07/30/2013 05:10:00 PM,Low,CD,*POLICE INFO,300 N PACA ST,21201.0,University Of Maryland,Central,121.0,11.0,D8,Downtown/Seton Hill,Census Tract 402,"300 N PACA ST\nBALTIMORE, MD\n(39.292866, -76...."
694919,5841881,132113173,07/30/2013 10:32:00 PM,Non-Emergency,CD,Transport,300 E MADISON ST,21202.0,Mount Vernon,Central,142.0,11.0,D8,Midtown,Census Tract 1101,"300 E MADISON ST\nBALTIMORE, MD\n(39.298812, -..."
694920,5842245,132112770,07/30/2013 08:34:00 PM,High,CD,REQUEST SERVICE,200 N CHARLES ST,21201.0,Downtown,Central,111.0,11.0,D7,Downtown/Seton Hill,Census Tract 401,"200 N CHARLES ST\nBALTIMORE, MD\n(39.291418, -..."
694921,5841680,132120009,07/31/2013 12:03:00 AM,Low,CD,Traffic Stop,1200 MC CULLOH ST,21217.0,Upton,Central,123.0,11.0,D8,Upton/Druid Heights,Census Tract 1702,"1200 MC CULLOH ST\nBALTIMORE, MD\n(39.301844, ..."


In [ ]:
# get test train

In [211]:
def get_test_2019_data(date):
    return date >= datetime.datetime(2019, 1, 1) and date <= datetime.datetime(2019, 12, 31)

def get_train_data(date):
    return date <= datetime.datetime(2018, 12, 31)

def get_test_train_split(df):
    train = df[df.index.to_series().apply(get_train_data)]
    train = train.rename(columns={"count":'y'})
    train["ds"] = train.index

    test = df[df.index.to_series().apply(get_test_2019_data)]
    test = test.rename(columns={"count":'y'})
    test["ds"] = test.index
    return(train, test)

In [214]:
train_d_SD, test_d_SD = get_test_train_split(calls_d_SD_df)

In [215]:
train_d_SW, test_d_SW = get_test_train_split(calls_d_SW_df)

In [216]:
train_d_SE, test_d_SE = get_test_train_split(calls_d_SE_df)

In [217]:
train_d_ND, test_d_ND = get_test_train_split(calls_d_ND_df)

In [218]:
train_d_NW, test_d_NW = get_test_train_split(calls_d_NW_df)

In [219]:
train_d_ED, test_d_ED = get_test_train_split(calls_d_ED_df)

In [220]:
train_d_CD, test_d_CD = get_test_train_split(calls_d_CD_df)

In [221]:
train_d_WD, test_d_WD = get_test_train_split(calls_d_WD_df)

In [222]:
train_d_NE, test_d_NE = get_test_train_split(calls_d_NE_df)

,y,ds
CallDateTime,,
2013-06-30,2,2013-06-30
2013-07-01,452,2013-07-01
2013-07-02,503,2013-07-02
2013-07-03,576,2013-07-03
2013-07-04,515,2013-07-04
...,...,...
2018-12-27,362,2018-12-27
2018-12-28,355,2018-12-28
2018-12-29,460,2018-12-29


In [ ]:
# get predictions

In [52]:
#df with test and train, test df, length of train df, len of test df
def moving_window_preds(df, test_df, window_length, test_len):
    preds_df = None
    
    min_idx = 0
    max_idx = window_length - 1
    
    df = df.rename(columns={"count":'y'})
    df["ds"] = df.index
    
    test_df = test_df.rename(columns={"count":'y'})
    test_df["ds"] = test_df.index

    for i in range(test_len):
        future_test_df = pd.DataFrame(((Prophet().fit(df[min_idx: max_idx])).predict(test_df)).iloc[i])
        if i == 0:
            preds_df = (future_test_df.transpose()).copy(deep = True)
        else:
            preds_df = preds_df.append(future_test_df.transpose())
        min_idx +=1
        max_idx +=1
    preds_df.set_index('ds').join(test_df.set_index('ds').y).reset_index()
    preds_df.dropna(inplace=True)
    return(preds_df)

In [53]:
def print_metrics(df):
    print(f'r2: {r2_score(df.y, df.yhat)}')
    print(f'MSE: {mean_squared_error(df.y, df.yhat)}')
    print(f'MAE: {mean_absolute_error(df.y, df.yhat)}')
    print(f'RMSE: {sqrt(mean_squared_error(df.y, df.yhat))}')

In [227]:
pred_2018_SD_d = moving_window_preds(calls_d_SD_df, test_d_SD, len(train_d_SD), len(test_d_SD))
pred_2018_SD_d = pred_2018_SD_d.set_index('ds').join(test_d_SD.set_index('ds').y).reset_index()
pred_2018_SD_d.dropna(inplace=True)
pred_2018_SD_d[['ds','y', 'yhat', 'yhat_lower', 'yhat_upper']].head()

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=Tru

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=Tru

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=Tru

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=Tru

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


,ds,y,yhat,yhat_lower,yhat_upper
0,2019-01-01,293,228.937,184.112,271.163
1,2019-01-02,325,227.89,185.969,274.438
2,2019-01-03,308,230.173,183.823,272.738
3,2019-01-04,302,246.548,204.539,293.595
4,2019-01-05,286,249.255,207.823,292.266


,y,ds
CallDateTime,,
2019-01-01,293,2019-01-01
2019-01-02,325,2019-01-02
2019-01-03,308,2019-01-03
2019-01-04,302,2019-01-04
2019-01-05,286,2019-01-05
...,...,...
2019-11-21,462,2019-11-21
2019-11-22,471,2019-11-22
2019-11-23,501,2019-11-23


In [228]:
pred_2018_SW_d = moving_window_preds(calls_d_SW_df, test_d_SW, len(train_d_SW), len(test_d_SW))
pred_2018_SW_d = pred_2018_SW_d.set_index('ds').join(test_d_SW.set_index('ds').y).reset_index()
pred_2018_SW_d.dropna(inplace=True)
pred_2018_SW_d[['ds','y', 'yhat', 'yhat_lower', 'yhat_upper']].head()

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=Tru

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=Tru

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=Tru

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=Tru

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


,ds,y,yhat,yhat_lower,yhat_upper
0,2019-01-01,318,268.468,223.796,315.298
1,2019-01-02,320,270.829,224.374,314.014
2,2019-01-03,287,274.076,231.829,322.864
3,2019-01-04,390,284.21,237.449,333.608
4,2019-01-05,387,267.141,221.52,311.131


In [229]:
pred_2018_SE_d = moving_window_preds(calls_d_SE_df, test_d_SE, len(train_d_SE), len(test_d_SE))
pred_2018_SE_d = pred_2018_SE_d.set_index('ds').join(test_d_SE.set_index('ds').y).reset_index()
pred_2018_SE_d.dropna(inplace=True)
pred_2018_SE_d[['ds','y', 'yhat', 'yhat_lower', 'yhat_upper']].head()

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=Tru

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=Tru

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=Tru

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=Tru

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


,ds,y,yhat,yhat_lower,yhat_upper
0,2019-01-01,397,256.178,213.551,300.71
1,2019-01-02,365,262.401,216.23,305.308
2,2019-01-03,276,268.623,225.042,310
3,2019-01-04,275,283.37,237.832,328.092
4,2019-01-05,268,282.803,234.194,325.832


In [230]:
pred_2018_ND_d = moving_window_preds(calls_d_ND_df, test_d_ND, len(train_d_ND), len(test_d_ND))
pred_2018_ND_d = pred_2018_ND_d.set_index('ds').join(test_d_ND.set_index('ds').y).reset_index()
pred_2018_ND_d.dropna(inplace=True)
pred_2018_ND_d[['ds','y', 'yhat', 'yhat_lower', 'yhat_upper']].head()

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=Tru

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=Tru

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=Tru

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=Tru

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


,ds,y,yhat,yhat_lower,yhat_upper
0,2019-01-01,243,226.992,182.666,270.812
1,2019-01-02,277,227.691,182.207,270.241
2,2019-01-03,271,229.42,186.664,269.478
3,2019-01-04,388,237.008,193.621,279.747
4,2019-01-05,336,219.7,175.544,265.488


In [231]:
pred_2018_NW_d = moving_window_preds(calls_d_NW_df, test_d_NW, len(train_d_NW), len(test_d_NW))
pred_2018_NW_d = pred_2018_NW_d.set_index('ds').join(test_d_NW.set_index('ds').y).reset_index()
pred_2018_NW_d.dropna(inplace=True)
pred_2018_NW_d[['ds','y', 'yhat', 'yhat_lower', 'yhat_upper']].head()

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=Tru

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=Tru

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=Tru

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=Tru

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


,ds,y,yhat,yhat_lower,yhat_upper
0,2019-01-01,235,221.94,179.34,258.539
1,2019-01-02,326,219.187,176.485,258.673
2,2019-01-03,279,221.963,180.378,262.445
3,2019-01-04,242,227.494,187.255,270.401
4,2019-01-05,213,213.637,167.884,253.959


In [232]:
pred_2018_ED_d = moving_window_preds(calls_d_ED_df, test_d_ED, len(train_d_ED), len(test_d_ED))
pred_2018_ED_d = pred_2018_ED_d.set_index('ds').join(test_d_ED.set_index('ds').y).reset_index()
pred_2018_ED_d.dropna(inplace=True)
pred_2018_ED_d[['ds','y', 'yhat', 'yhat_lower', 'yhat_upper']].head()

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=Tru

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=Tru

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=Tru

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=Tru

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


,ds,y,yhat,yhat_lower,yhat_upper
0,2019-01-01,248,202.896,162.187,242.694
1,2019-01-02,260,204.975,165.443,245.736
2,2019-01-03,269,207.188,165.556,246.386
3,2019-01-04,276,217.029,174.229,256.325
4,2019-01-05,199,193.048,154.312,233.894


In [233]:
pred_2018_CD_d = moving_window_preds(calls_d_CD_df, test_d_CD, len(train_d_CD), len(test_d_CD))
pred_2018_CD_d = pred_2018_CD_d.set_index('ds').join(test_d_CD.set_index('ds').y).reset_index()
pred_2018_CD_d.dropna(inplace=True)
pred_2018_CD_d[['ds','y', 'yhat', 'yhat_lower', 'yhat_upper']].head()

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=Tru

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=Tru

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=Tru

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=Tru

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


,ds,y,yhat,yhat_lower,yhat_upper
0,2019-01-01,380,238.661,194.516,279.592
1,2019-01-02,273,245.108,202.823,290.628
2,2019-01-03,319,251.898,205.003,294.811
3,2019-01-04,356,268.667,226.146,310.823
4,2019-01-05,332,237.582,194.738,277.356


In [234]:
pred_2018_WD_d = moving_window_preds(calls_d_WD_df, test_d_WD, len(train_d_WD), len(test_d_WD))
pred_2018_WD_d = pred_2018_WD_d.set_index('ds').join(test_d_WD.set_index('ds').y).reset_index()
pred_2018_WD_d.dropna(inplace=True)
pred_2018_WD_d[['ds','y', 'yhat', 'yhat_lower', 'yhat_upper']].head()

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=Tru

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=Tru

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=Tru

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=Tru

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


,ds,y,yhat,yhat_lower,yhat_upper
0,2019-01-01,307,254.823,210.61,299.634
1,2019-01-02,350,258.62,214.003,306.176
2,2019-01-03,337,259.852,215.253,306.806
3,2019-01-04,374,273.319,228.809,314.044
4,2019-01-05,337,260.006,215.301,303.166


In [235]:
pred_2018_NE_d = moving_window_preds(calls_d_NE_df, test_d_NE, len(train_d_NE), len(test_d_NE))
pred_2018_NE_d = pred_2018_NE_d.set_index('ds').join(test_d_NE.set_index('ds').y).reset_index()
pred_2018_NE_d.dropna(inplace=True)
pred_2018_NE_d[['ds','y', 'yhat', 'yhat_lower', 'yhat_upper']].head()

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=Tru

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=Tru

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=Tru

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=Tru

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


,ds,y,yhat,yhat_lower,yhat_upper
0,2019-01-01,372,302.38,246.146,352.688
1,2019-01-02,346,304.04,250.201,355.589
2,2019-01-03,423,302.461,250.484,351.51
3,2019-01-04,441,314.249,265.605,363.208
4,2019-01-05,411,306.542,256.934,356.257


In [250]:
calls_d_SD_df


,count
CallDateTime,
2013-07-01,426
2013-07-02,316
2013-07-03,352
2013-07-04,371
2013-07-05,393
...,...
2019-11-21,462
2019-11-22,471
2019-11-23,501


In [ ]:
# forecast to tableau

In [236]:
forecast_dict = {'SD': (pred_2018_SD_d, calls_d_SD_df),
                'SW': (pred_2018_SW_d, calls_d_SW_df),
                'SE': (pred_2018_SE_d, calls_d_SE_df),
                'ND': (pred_2018_ND_d, calls_d_ND_df),
                'NW': (pred_2018_NW_d, calls_d_NW_df),
                'ED': (pred_2018_ED_d, calls_d_ED_df),
                'CD': (pred_2018_CD_d, calls_d_CD_df),
                'WD': (pred_2018_WD_d, calls_d_WD_df),
                'NE': (pred_2018_NE_d, calls_d_NE_df)}

In [253]:
def get_forecast(Date, District):
    #may need to convert datetime as df
    date = datetime.datetime.strptime(Date, '%m/%d/%Y')
    d64 = np.datetime64(date)
    if d64 in forecast_dict[District][0].ds.values:  
        df = forecast_dict[District][0]
        return ([str(int(df.loc[df['ds'] == d64, 'yhat']))])
    elif d64 in forecast_dict[District][1].index.values:  
        df = forecast_dict[District][1]
        return ([str(int(df.loc[df.index == d64, 'count']))])
    else:
        return ("Invalid date; No data for this date")

In [254]:
Date = "01/02/2018"
District = "SW"



# date = datetime.strptime("2018-01-01", '%Y-%m-%d')#'%m-%d-%Y')
# now = date.now()
# timestamp = datetime.timestamp(now)
#date = datetime.strptime(Date, '%m/%d/%Y')
#d64 = np.datetime64(date)
#df = forecast_dict[District]
#df.loc[df['ds'] == d64, 'count']
get_forecast(Date, District)

['231']

In [195]:
if d64 in pred_2018_CD_d.ds.values:
    print('hi')
calls_d_CD_df

hi


,count
CallDateTime,
2013-07-01,416
2013-07-02,407
2013-07-03,431
2013-07-04,375
2013-07-05,371
...,...
2019-11-21,415
2019-11-22,362
2019-11-23,443


In [123]:

df.loc[df['ds'] == d64, 'yhat']
d64

numpy.datetime64('2018-01-02T00:00:00.000000')

In [90]:

type(df.ds.iloc[0])

pandas._libs.tslibs.timestamps.Timestamp

In [87]:
type(df)

pandas._libs.tslibs.timestamps.Timestamp

In [88]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 365 entries, 0 to 364
Data columns (total 20 columns):
ds                            365 non-null datetime64[ns]
trend                         365 non-null object
yhat_lower                    365 non-null object
yhat_upper                    365 non-null object
trend_lower                   365 non-null object
trend_upper                   365 non-null object
additive_terms                365 non-null object
additive_terms_lower          365 non-null object
additive_terms_upper          365 non-null object
weekly                        365 non-null object
weekly_lower                  365 non-null object
weekly_upper                  365 non-null object
yearly                        365 non-null object
yearly_lower                  365 non-null object
yearly_upper                  365 non-null object
multiplicative_terms          365 non-null object
multiplicative_terms_lower    365 non-null object
multiplicative_terms_upper    365 non

In [175]:
# Connect to TabPy server using the client library
connection = tabpy_client.Client('http://localhost:9004/')

In [256]:
# Publish the SuggestDiagnosis function to TabPy server so it can be used from Tableau
# Using the name DiagnosticsDemo and a short description of what it does
connection.deploy('Forecast',
                  get_forecast,
                  'Returns forecast for given date and District')

INFO:tabpy_client.rest:GET http://localhost:9004/endpoints with {}
INFO:tabpy_client.rest:response='{}'
INFO:tabpy_client.rest:GET http://localhost:9004/configurations/endpoint_upload_destination with None
INFO:tabpy_client.rest:response='{"path": "/opt/anaconda3/envs/metis/lib/python3.7/site-packages/tabpy_server/staging"}'
INFO:tabpy_client.rest:Initializing ModelEndpoint from {'name': 'Forecast', 'version': 1, 'description': 'Returns forecast for given date and District', 'type': 'model', 'endpoint_obj': <tabpy_client.custom_query_object.CustomQueryObject object at 0x145c22198>, 'dependencies': [], 'methods': [{'method': 'query'}], 'required_files': [], 'required_packages': [], 'schema': None, 'src_path': '/opt/anaconda3/envs/metis/lib/python3.7/site-packages/tabpy_server/staging/endpoints/Forecast/1'}
INFO:tabpy_client.rest:POST http://localhost:9004/endpoints with '{"required_files": [], "methods": [{"method": "query"}], "description": "Returns forecast for given date and District

In [255]:
#connection.remove('Forecast')

INFO:tabpy_client.rest:DELETE http://localhost:9004/endpoints/Forecast with None
INFO:tabpy_client.rest:GET http://localhost:9004/endpoints with {}
INFO:tabpy_client.rest:response='{}'
